# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 3 2023 1:59PM,254195,15,VT80182204,"Virtus Pharmaceuticals, LLC",Released
1,Jan 3 2023 1:59PM,254195,15,VT80182205,"Virtus Pharmaceuticals, LLC",Released
2,Jan 3 2023 1:59PM,254195,15,VT80182206,"Virtus Pharmaceuticals, LLC",Released
3,Jan 3 2023 1:59PM,254195,15,VT80182207,"Virtus Pharmaceuticals, LLC",Released
4,Jan 3 2023 1:59PM,254195,15,VT80182208,"Virtus Pharmaceuticals, LLC",Released
5,Jan 3 2023 1:59PM,254195,15,VT80182209,"Virtus Pharmaceuticals, LLC",Released
6,Jan 3 2023 1:59PM,254195,15,VT80182210,"Virtus Pharmaceuticals, LLC",Released
7,Jan 3 2023 1:59PM,254195,15,VT80182211,"Virtus Pharmaceuticals, LLC",Released
8,Jan 3 2023 1:59PM,254195,15,VT80182236,"Virtus Pharmaceuticals, LLC",Released
9,Jan 3 2023 1:59PM,254195,15,VT80182237,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,254189,Released,29
47,254190,Released,1
48,254192,Released,1
49,254193,Released,1
50,254195,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254189,NaN,NaN,29.0
254190,NaN,NaN,1.0
254192,NaN,NaN,1.0
254193,NaN,NaN,1.0
254195,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,44.0,15.0,3.0
254104,0.0,0.0,1.0
254108,0.0,1.0,23.0
254111,0.0,0.0,53.0
254117,0.0,37.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,44,15,3
254104,0,0,1
254108,0,1,23
254111,0,0,53
254117,0,37,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,44,15,3
1,254104,0,0,1
2,254108,0,1,23
3,254111,0,0,53
4,254117,0,37,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,44,15,3
1,254104,,,1
2,254108,,1,23
3,254111,,,53
4,254117,,37,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC"
11,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated"
12,Jan 3 2023 1:36PM,254192,10,Emerginnova
13,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation
42,Jan 3 2023 1:24PM,254190,19,"Zymergen, Inc."
43,Jan 3 2023 1:18PM,254175,10,"Citieffe, Inc."
44,Jan 3 2023 1:15PM,254186,10,"Methapharm, Inc."
60,Jan 3 2023 1:13PM,254188,10,Yusen – 3D Matrix
62,Jan 3 2023 1:13PM,254187,10,ISDIN Corporation
65,Jan 3 2023 1:11PM,254185,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",,,11
1,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated",,,1
2,Jan 3 2023 1:36PM,254192,10,Emerginnova,,,1
3,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation,,,29
4,Jan 3 2023 1:24PM,254190,19,"Zymergen, Inc.",,,1
5,Jan 3 2023 1:18PM,254175,10,"Citieffe, Inc.",,,1
6,Jan 3 2023 1:15PM,254186,10,"Methapharm, Inc.",,,16
7,Jan 3 2023 1:13PM,254188,10,Yusen – 3D Matrix,,,2
8,Jan 3 2023 1:13PM,254187,10,ISDIN Corporation,,,3
9,Jan 3 2023 1:11PM,254185,10,Emerginnova,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",11,,
1,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated",1,,
2,Jan 3 2023 1:36PM,254192,10,Emerginnova,1,,
3,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation,29,,
4,Jan 3 2023 1:24PM,254190,19,"Zymergen, Inc.",1,,
5,Jan 3 2023 1:18PM,254175,10,"Citieffe, Inc.",1,,
6,Jan 3 2023 1:15PM,254186,10,"Methapharm, Inc.",16,,
7,Jan 3 2023 1:13PM,254188,10,Yusen – 3D Matrix,2,,
8,Jan 3 2023 1:13PM,254187,10,ISDIN Corporation,3,,
9,Jan 3 2023 1:11PM,254185,10,Emerginnova,3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",11,,
1,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated",1,,
2,Jan 3 2023 1:36PM,254192,10,Emerginnova,1,,
3,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation,29,,
4,Jan 3 2023 1:24PM,254190,19,"Zymergen, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",11.0,NaN,NaN
1,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Jan 3 2023 1:36PM,254192,10,Emerginnova,1.0,NaN,NaN
3,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation,29.0,NaN,NaN
4,Jan 3 2023 1:24PM,254190,19,"Zymergen, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",11.0,0.0,0.0
1,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Jan 3 2023 1:36PM,254192,10,Emerginnova,1.0,0.0,0.0
3,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation,29.0,0.0,0.0
4,Jan 3 2023 1:24PM,254190,19,"Zymergen, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7370451,639.0,74.0,0.0
12,762427,1.0,21.0,0.0
15,1016697,126.0,0.0,0.0
19,1524952,11.0,20.0,44.0
20,508253,2.0,0.0,0.0
21,254144,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7370451,639.0,74.0,0.0
1,12,762427,1.0,21.0,0.0
2,15,1016697,126.0,0.0,0.0
3,19,1524952,11.0,20.0,44.0
4,20,508253,2.0,0.0,0.0
5,21,254144,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,639.0,74.0,0.0
1,12,1.0,21.0,0.0
2,15,126.0,0.0,0.0
3,19,11.0,20.0,44.0
4,20,2.0,0.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,639.0
1,12,Released,1.0
2,15,Released,126.0
3,19,Released,11.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,44.0,0.0,0.0
Executing,74.0,21.0,0.0,20.0,0.0,0.0
Released,639.0,1.0,126.0,11.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,0.0,44.0,0.0,0.0
1,Executing,74.0,21.0,0.0,20.0,0.0,0.0
2,Released,639.0,1.0,126.0,11.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,0.0,44.0,0.0,0.0
1,Executing,74.0,21.0,0.0,20.0,0.0,0.0
2,Released,639.0,1.0,126.0,11.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()